<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=5f705bad4d6f99ff2fea559cc5e8d8be02fa8428931bd8e0976c95648f8566ab
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1


    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-03 16:12:12
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.44 C
-------------------
Today PnL: -79.93 K (-0.55%)
Current PnL: -24.51 L (-15.83%)
CY Booked + Current PnL: -10.22 L (-6.6%)
-------------------
Total profit:  1.19 L
Total loss:  -25.70 L
-------------------
Total Booked + Current PnL: 16.76 L (13.11%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (9.94%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 89.14 L (62.0%)
Deployed:  1.28 C
Current:  1.44 C
CAGR/XIRR %: 6.71%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.51,-8.68,9.52,0.02,15176.0,-15145.0,159414.0,143.30,66.0,M-SC,14.27,234.0,-1.00,1.15,38.66,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.19,-16.71,20.17,0.09,16930.0,-16839.0,83938.0,94.88,37.0,M-SC,2.93,253.0,-0.99,0.60,10.28,OX40N,NTT,DURABLES
50,MASFIN,398.61,0.57,-2.01,24.55,22.05,23572.0,-1965.0,96015.0,-15.54,60.0,H-SC,6.74,164.0,-0.08,0.69,39.56,XR,ATH,FINANCE
43,ITC,452.00,-0.11,-2.15,12.86,10.44,25186.0,-4294.0,195844.0,-41.56,40.0,X-LC,1.51,5.0,-0.17,1.41,3.30,X40,NTT,FMCG
35,ICICIGI,2252.93,-0.72,5.23,14.22,20.19,26084.0,9113.0,183433.0,-17.43,45.0,X-MC,6.72,68.0,0.35,1.32,21.13,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,ROUTE,2227.21,5.40,-46.54,216.23,69.05,153813.0,-61932.0,71134.0,-56.81,54.0,H-SC,23.77,140.0,-0.40,0.51,9.70,SR,ATH,IT
13,BSOFT,831.70,4.40,-23.36,96.74,50.79,104287.0,-32852.0,107801.0,-1.82,72.0,H-SC,5.88,151.0,-0.32,0.78,25.52,XR,ATH,IT
48,KPIGREEN,730.66,3.86,-10.20,63.64,46.95,71606.0,-12779.0,112518.0,-16.98,42.0,H-SC,2.36,154.0,-0.18,0.81,40.96,MH,ATH,POWER
84,WIPRO,420.00,1.81,4.46,64.91,72.27,114897.0,7562.0,177010.0,-10.16,67.0,M-LC,3.13,99.0,0.07,1.28,10.91,XR,NTT,IT
16,CERA,9475.00,1.42,-28.26,72.70,23.89,91740.0,-49713.0,126190.0,-30.79,34.0,H-SC,7.27,157.0,-0.54,0.91,8.98,OX40N,NTT,CERAMICS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-5.08,-39.32,86.07,12.90,114111.0,-85917.0,132579.0,95.91,46.0,H-SC,12.17,161.0,-0.75,0.96,10.24,OX40N,NTT,FINANCE
24,EASEMYTRIP,26.40,-3.98,-54.75,221.95,45.70,177596.0,-96799.0,80016.0,-16.85,61.0,M-SC,23.43,205.0,-0.55,0.58,15.33,XY24,NTT,TRAVEL
22,DIXON,18931.72,-3.28,-9.97,35.05,21.58,44220.0,-13977.0,126162.0,-54.61,28.0,X-MC,5.88,56.0,-0.32,0.91,11.69,X40N,ATH,IT
67,SONACOMS,806.63,-2.91,-14.95,64.07,39.54,55125.0,-15120.0,86039.0,-33.43,50.0,M-SC,6.41,220.0,-0.27,0.62,21.32,AR,ATH,AUTO
83,WHIRLPOOL,2270.00,-2.72,-21.27,131.07,81.92,118462.0,-24417.0,90381.0,-53.88,22.0,M-SC,4.96,236.0,-0.21,0.65,7.00,XR,NTT,DURABLES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,0.15,0.1,106.51,106.72,131502.0,128.0,123464.0,-50.44,40.0,H-SC,1.72,139.0,0.0,0.89,23.51,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.41,-6.05,121.29,107.91,168250.0,-8927.0,138717.0,-23.39,32.0,M-SC,11.15,216.0,-0.05,1.00,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.51,-8.68,9.52,0.02,15176.0,-15145.0,159414.0,143.30,66.0,M-SC,14.27,234.0,-1.00,1.15,38.66,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.19,-16.71,20.17,0.09,16930.0,-16839.0,83938.0,94.88,37.0,M-SC,2.93,253.0,-0.99,0.60,10.28,OX40N,NTT,DURABLES
47,KANSAINER,340.00,0.88,-22.35,46.13,13.47,96598.0,-60264.0,209403.0,-68.12,36.0,H-SC,2.12,158.0,-0.62,1.51,6.48,XY24,NTT,PAINTS
66,SIS,528.00,1.09,-24.46,60.54,21.26,50775.0,-27162.0,83870.0,1978.24,47.0,H-SC,3.91,166.0,-0.53,0.60,13.32,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,1.81,4.46,64.91,72.27,114897.0,7562.0,177010.0,-10.16,67.0,M-LC,3.13,99.0,0.07,1.28,10.91,XR,NTT,IT
37,IEX,219.00,0.24,1.05,47.09,48.63,95143.0,2090.0,202044.0,-32.69,66.0,H-SC,14.78,136.0,0.02,1.46,12.72,XR,NTT,MISC
38,INDIAMART,4810.62,0.15,0.10,106.51,106.72,131502.0,128.0,123464.0,-50.44,40.0,H-SC,1.72,139.0,0.00,0.89,23.51,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,1.81,4.46,64.91,72.27,114897.0,7562.0,177010.0,-10.16,67.0,M-LC,3.13,99.0,0.07,1.28,10.91,XR,NTT,IT
37,IEX,219.00,0.24,1.05,47.09,48.63,95143.0,2090.0,202044.0,-32.69,66.0,H-SC,14.78,136.0,0.02,1.46,12.72,XR,NTT,MISC
38,INDIAMART,4810.62,0.15,0.10,106.51,106.72,131502.0,128.0,123464.0,-50.44,40.0,H-SC,1.72,139.0,0.00,0.89,23.51,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,-0.82,-1.41,37.57,35.63,77629.0,-2950.0,206624.0,-15.41,42.0,H-MC,3.64,119.0,-0.04,1.49,15.03,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.41,-6.05,121.29,107.91,168250.0,-8927.0,138717.0,-23.39,32.0,M-SC,11.15,216.0,-0.05,1.00,0.00,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.05,-20.69,87.17,48.45,138627.0,-41475.0,159031.0,-26.32,13.0,X-MC,8.85,64.0,-0.30,1.15,0.00,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,1.32,-36.95,114.44,35.20,92843.0,-47542.0,81128.0,6.43,26.0,X-SC,14.19,92.0,-0.51,0.58,1.32,X40,NTT,FOOTWEAR
8,AWL,485.00,-0.74,-21.65,95.92,53.50,226764.0,-65341.0,236410.0,-62.10,28.0,X-MC,1.96,58.0,-0.29,1.70,5.01,XY24,NTT,FMCG
22,DIXON,18931.72,-3.28,-9.97,35.05,21.58,44220.0,-13977.0,126162.0,-54.61,28.0,X-MC,5.88,56.0,-0.32,0.91,11.69,X40N,ATH,IT
23,DMART,5391.45,-1.32,-7.09,37.90,28.12,74089.0,-14922.0,195485.0,-19.86,29.0,X-LC,4.35,19.0,-0.20,1.41,14.86,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.0,-0.63,-22.66,112.26,64.15,206581.0,-53931.0,184020.0,-54.57,46.0,X-SC,1.45,82.0,-0.26,1.33,3.23,XY24,BTT,CEMENT
43,ITC,452.0,-0.11,-2.15,12.86,10.44,25186.0,-4294.0,195844.0,-41.56,40.0,X-LC,1.51,5.0,-0.17,1.41,3.30,X40,NTT,FMCG
15,CAMS,4762.0,0.23,1.14,23.38,24.79,59557.0,2878.0,254734.0,-3.47,47.0,X-SC,1.91,86.0,0.05,1.84,24.55,X40N,NTT,MISC
8,AWL,485.0,-0.74,-21.65,95.92,53.50,226764.0,-65341.0,236410.0,-62.10,28.0,X-MC,1.96,58.0,-0.29,1.70,5.01,XY24,NTT,FMCG
36,ICICIPRULI,790.0,-0.86,1.33,29.24,30.97,52368.0,2358.0,179096.0,-22.52,47.0,X-MC,2.18,75.0,0.05,1.29,14.06,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.05,-20.69,87.17,48.45,138627.0,-41475.0,159031.0,-26.32,13.0,X-MC,8.85,64.0,-0.30,1.15,0.0,X40N,ATH,FINANCE
19,COLPAL,3726.84,-2.61,-20.19,77.31,41.51,162498.0,-53175.0,210190.0,-4.98,30.0,X-MC,7.32,61.0,-0.33,1.51,0.0,XY25,ATH,FMCG
55,QUESS,424.00,-0.93,-30.12,104.41,42.84,47430.0,-19579.0,45427.0,-54.27,37.0,X-SC,37.42,83.0,-0.41,0.33,0.0,XY24,NTT,MISC
53,PAGEIND,51605.08,-0.97,-9.03,38.57,26.07,57454.0,-14780.0,148960.0,-33.02,29.0,X-MC,10.49,55.0,-0.26,1.07,0.0,X40,ATH,APPARELS
54,PGHH,17907.65,-0.48,-5.59,41.69,33.76,79038.0,-11235.0,189585.0,-33.45,29.0,X-MC,4.52,57.0,-0.14,1.37,0.0,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.93,-30.12,104.41,42.84,47430.0,-19579.0,45427.0,-54.27,37.0,X-SC,37.42,83.0,-0.41,0.33,0.00,XY24,NTT,MISC
59,RELAXO,1176.00,-0.02,-47.40,189.55,52.31,144733.0,-68804.0,76356.0,-43.96,35.0,X-SC,5.01,91.0,-0.48,0.55,2.21,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,1.32,-36.95,114.44,35.20,92843.0,-47542.0,81128.0,6.43,26.0,X-SC,14.19,92.0,-0.51,0.58,1.32,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-3.28,-9.97,35.05,21.58,44220.0,-13977.0,126162.0,-54.61,28.0,X-MC,5.88,56.0,-0.32,0.91,11.69,X40N,ATH,IT
51,MEDANTA,1486.00,-1.00,1.23,22.57,24.08,29006.0,1564.0,128514.0,-6.93,41.0,X-SC,5.82,89.0,0.05,0.93,21.09,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,1.41,-13.60,38.05,19.28,113739.0,-47036.0,298920.0,-24.60,64.0,X-LC,6.73,1.0,-0.41,2.15,10.51,X40,ATH,IT
41,INFY,2275.00,1.13,7.47,44.11,54.87,148326.0,23375.0,336263.0,-16.37,64.0,X-LC,2.71,2.0,0.16,2.42,16.37,X40,BTT,IT
75,TMPV,600.00,-1.35,-15.44,68.14,42.18,159268.0,-42673.0,233737.0,-24.30,32.0,X-LC,3.13,3.0,-0.27,1.68,1.25,XY24,NTT,AUTO
81,VBL,671.64,-0.81,-3.55,40.66,35.67,123869.0,-11197.0,304645.0,-14.99,58.0,X-LC,5.39,4.0,-0.09,2.19,9.82,X40N,ATH,FMCG
43,ITC,452.00,-0.11,-2.15,12.86,10.44,25186.0,-4294.0,195844.0,-41.56,40.0,X-LC,1.51,5.0,-0.17,1.41,3.30,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-0.22,-15.08,113.93,81.67,90763.0,-14144.0,79666.0,7161.11,44.0,L-SC,18.76,271.0,-0.16,0.57,54.28,XR,NTT,CERAMICS
50,MASFIN,398.61,0.57,-2.01,24.55,22.05,23572.0,-1965.0,96015.0,-15.54,60.0,H-SC,6.74,164.0,-0.08,0.69,39.56,XR,ATH,FINANCE
13,BSOFT,831.70,4.40,-23.36,96.74,50.79,104287.0,-32852.0,107801.0,-1.82,72.0,H-SC,5.88,151.0,-0.32,0.78,25.52,XR,ATH,IT
69,SURYODAY,216.00,-0.96,-19.74,53.29,23.03,76593.0,-35343.0,143728.0,55.70,48.0,H-SC,9.98,167.0,-0.46,1.04,42.61,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,0.51,-8.68,9.52,0.02,15176.0,-15145.0,159414.0,143.30,66.0,M-SC,14.27,234.0,-1.00,1.15,38.66,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.51,-8.68,9.52,0.02,15176.0,-15145.0,159414.0,143.30,66.0,M-SC,14.27,234.0,-1.00,1.15,38.66,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.03,3.23,17.16,20.95,44600.0,8140.0,259908.0,2.18,79.0,X-LC,17.34,31.0,0.18,1.87,40.02,X40,ATH,PAINTS
13,BSOFT,831.70,4.40,-23.36,96.74,50.79,104287.0,-32852.0,107801.0,-1.82,72.0,H-SC,5.88,151.0,-0.32,0.78,25.52,XR,ATH,IT
65,SIEMENS,4671.50,-0.93,-10.53,40.29,25.51,67081.0,-19600.0,166495.0,2.20,64.0,H-LC,2.70,49.0,-0.29,1.20,22.69,AR,ATH,ELECTRICAL
37,IEX,219.00,0.24,1.05,47.09,48.63,95143.0,2090.0,202044.0,-32.69,66.0,H-SC,14.78,136.0,0.02,1.46,12.72,XR,NTT,MISC


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.28
1,25,45.13
2,50,76.13


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.37
MC    29.45
LC    24.18
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.15
X40      21.78
X40N     12.13
XR        9.93
AR        9.25
XY25      9.19
OX40N     7.74
SR        1.02
MH        0.81
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.43
X-MC    22.87
X-LC    20.70
M-SC    12.16
X-SC     8.37
H-MC     4.84
L-SC     1.41
M-MC     1.39
M-LC     1.28
H-LC     1.20
L-LC     1.00
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.01,-6.51,40.96
IT,12.42,-16.88,80.07
FINANCE,9.48,-17.81,67.58
MISC,7.62,-22.93,73.30
PAINTS,6.16,-8.01,24.06
ELECTRICAL,6.13,-10.43,50.88
INSURANCE,4.46,-2.12,38.11
PHARMA,3.98,-1.11,33.47
AUTO,3.41,-21.40,73.46


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3129943.0,21
AR,1288468.0,10
XR,1285453.0,13
X40,1013565.0,14
X40N,765373.0,9
OX40N,717758.0,10
XY25,363803.0,6
SR,277994.0,2
MH,71606.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3587493.0,25
M-SC,1371966.0,15
X-MC,1306841.0,16
X-LC,886241.0,11
X-SC,759851.0,8
H-MC,407144.0,3
L-SC,264945.0,3
M-LC,114897.0,1
H-LC,67081.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1251735.0      6
           AR         883806.0      5
M-SC       XY24       781030.0      6
H-SC       XR         769983.0      7
X-MC       X40        458438.0      7
           XY24       387129.0      3
X-LC       X40        376048.0      5
H-SC       OX40N      332369.0      4
M-SC       OX40N      299124.0      5
X-SC       X40N       297755.0      3
           XY24       283017.0      3
H-SC       SR         277994.0      2
X-MC       X40N       269660.0      4
X-LC       XY24       233224.0      2
H-MC       AR         213336.0      2
X-LC       X40N       197958.0      2
H-MC       XY24       193808.0      1
X-MC       XY25       191614.0      2
X-SC       X40        179079.0      2
L-SC       XR         178680.0      2
M-SC       XR         167567.0      2
           AR         124245.0      2
M-LC       XR         114897.0      1
L-SC       OX40N       86265.0      1
X-LC       XY25        79011.0      2
H-SC       MH          71606.0      1
H-LC       AR          67081.0      1
L-MC       XR          54326.0      1
M-MC       XY25        52636.0      1
L-LC       XY25        40542.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 49.0 seconds
